In [2]:
# Loading my OPENAI KEY as an environment variable
from dotenv import load_dotenv
load_dotenv()

True

## Audio transcription

We begin by  transcribing the audio from the input

In [7]:
import whisper
import os

audio_input_path = "audio/audio_to_transcribe.mp3" 

model = whisper.load_model("base.en")

In [8]:
result = model.transcribe(audio_input_path)
print(result["text"])

 Today we are going to be looking at how to use the VITN application to view your exam schedule. We will begin by opening the VITN application. Once it is opened up, we tap on the refresh button in order to get the latest data and keep it updated. Then we open the sidebar by tapping on the sidebar icon, go into academics, down and exam schedule. From here we can select our exam and now view the exam venue, see details, time and date. Thank you.


In [62]:
transcript = result["text"]

In [30]:
print(result)

{'text': ' Today we are going to be looking at how to use the VITN application to view your exam schedule. We will begin by opening the VITN application. Once it is opened up, we tap on the refresh button in order to get the latest data and keep it updated. Then we open the sidebar by tapping on the sidebar icon, go into academics, down and exam schedule. From here we can select our exam and now view the exam venue, see details, time and date. Thank you.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.8000000000000003, 'text': ' Today we are going to be looking at how to use the VITN application to view your exam', 'tokens': [50363, 6288, 356, 389, 1016, 284, 307, 2045, 379, 703, 284, 779, 262, 569, 2043, 45, 3586, 284, 1570, 534, 2814, 50553], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 1, 'seek': 0, 'start': 3.8000000000000003, 'end': 4.8, 'text': ' schedule.', 'tokens': [5

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

## Steps generation

1. We generate steps taken in the video, alongside the timestamps for when the step begins.
2. We will use the start time of each step to display the step_text in our video

In [71]:
# Prompt template to generate the narrator text
topic_template = """
You are given the transcript of a video of me using an app.
You are to convert the given text of actions performed in the video into an organized list of steps and properly number them.
You are to also timestamp each of the steps as per the transcript. 
The output must be a python list of tuples like [(step_text, timestamp), (step_text, timestamp)] where each step_text is a step and timestamp is the start of the text in seconds . Be informative and do not make up things.

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The Transcript:
```
{transcript}
```
Do not give any additional text. No talk, just go. 
"""

In [72]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are an expert at making tutorial videos and very good at defining tasks in simple terms.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc","tutorial_desc", "transcript"]))

In [73]:
# Chain to write the create the steps from the input text
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
steps_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='steps')

In [74]:
# Description of the app we are using

# USER INPUT
app_desc = """VITian is an application built for students of VIT to access their student data. 
We can use the app to find out examination allotments."""

# Tutorial Description

#USER INPUT
tutorial_desc = """This is a video tutorial on how to use the VITian app to check exam schedule and timings."""

# transcript has been acquired from the whisper module

In [75]:
# testing the steps chain + prompt engineering
op = steps_chain(inputs={"app_desc": app_desc, "tutorial_desc": tutorial_desc, "transcript": transcript}, return_only_outputs=True)

In [76]:
print(op)

{'steps': 'Here is the organized list of steps with timestamps:\n\n```python\n[\n    ("Open the VITian application.", 0),\n    ("Tap on the refresh button to update the data.", 10),\n    ("Open the sidebar by tapping on the sidebar icon.", 20),\n    ("Navigate to \'Academics\' in the sidebar.", 25),\n    ("Scroll down and select \'Exam Schedule\'.", 30),\n    ("Select your exam.", 35),\n    ("View the exam venue, details, time, and date.", 40)\n]\n```'}


In [25]:
# testing the steps + prompt engineering

In [48]:
# messages = [
#     SystemMessage(
#         content="You are a narrator with experience in making tech tutorial videos."
#     ),
#     HumanMessage(
#         content="""

# You are given the transcript of a video of me using an app.
# You are to convert the given text of actions performed in the video into an organized list of steps and properly number them.
# You are to also timestamp each of the steps as per the transcript. 
# The output must be a python list of tuples like [(step_text, timestamp), (step_text, timestamp)] where each step_text is a step and timestamp is the start of the text in seconds . Be informative and do not make up things.

# App details:
# ```
# VITian is an application built for students of VIT to access their student data. 
# We can use the app to find out examination allotments.
# ```

# Tutorial Description:
# ```
# This is a video tutorial on how to use the VITian app to check exam schedule and timings.
# ```

# The Transcript:
# ```
# {'text': ' Today we are going to be looking at how to use the VITN application to view your exam schedule. We will begin by opening the VITN application. Once it is opened up, we tap on the refresh button in order to get the latest data and keep it updated. Then we open the sidebar by tapping on the sidebar icon, go into academics, down and exam schedule. From here we can select our exam and now view the exam venue, see details, time and date. Thank you.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.8000000000000003, 'text': ' Today we are going to be looking at how to use the VITN application to view your exam', 'tokens': [50363, 6288, 356, 389, 1016, 284, 307, 2045, 379, 703, 284, 779, 262, 569, 2043, 45, 3586, 284, 1570, 534, 2814, 50553], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 1, 'seek': 0, 'start': 3.8000000000000003, 'end': 4.8, 'text': ' schedule.', 'tokens': [50553, 7269, 13, 50603], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 2, 'seek': 0, 'start': 4.8, 'end': 9.08, 'text': ' We will begin by opening the VITN application.', 'tokens': [50603, 775, 481, 2221, 416, 4756, 262, 569, 2043, 45, 3586, 13, 50817], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 3, 'seek': 0, 'start': 9.08, 'end': 13.4, 'text': ' Once it is opened up, we tap on the refresh button in order to get the latest data and', 'tokens': [50817, 4874, 340, 318, 4721, 510, 11, 356, 9814, 319, 262, 14976, 4936, 287, 1502, 284, 651, 262, 3452, 1366, 290, 51033], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 4, 'seek': 0, 'start': 13.4, 'end': 15.200000000000001, 'text': ' keep it updated.', 'tokens': [51033, 1394, 340, 6153, 13, 51123], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 5, 'seek': 0, 'start': 15.200000000000001, 'end': 21.04, 'text': ' Then we open the sidebar by tapping on the sidebar icon, go into academics, down and', 'tokens': [51123, 3244, 356, 1280, 262, 40217, 416, 24435, 319, 262, 40217, 7196, 11, 467, 656, 22321, 11, 866, 290, 51415], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 6, 'seek': 0, 'start': 21.04, 'end': 23.400000000000002, 'text': ' exam schedule.', 'tokens': [51415, 2814, 7269, 13, 51533], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 7, 'seek': 0, 'start': 23.400000000000002, 'end': 29.2, 'text': ' From here we can select our exam and now view the exam venue, see details, time and date.', 'tokens': [51533, 3574, 994, 356, 460, 2922, 674, 2814, 290, 783, 1570, 262, 2814, 14359, 11, 766, 3307, 11, 640, 290, 3128, 13, 51823], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 8, 'seek': 2920, 'start': 29.2, 'end': 30.2, 'text': ' Thank you.', 'tokens': [50363, 6952, 345, 13, 50413], 'temperature': 0.0, 'avg_logprob': -0.9295773506164551, 'compression_ratio': 0.5555555555555556, 'no_speech_prob': 0.996982991695404}], 'language': 'en'}
# ```
# ```
# """
#     ),
# ]

In [43]:
# op = chat(messages)

In [51]:
# print(op.content)

Here is the organized list of steps with timestamps:

```python
[
    ("Open the VITN application", 4.8),
    ("Tap on the refresh button to get the latest data", 9.08),
    ("Open the sidebar by tapping on the sidebar icon", 15.2),
    ("Go into academics, then exam schedule", 21.04),
    ("Select your exam to view the exam venue, details, time and date", 23.4)
]
```


## Narrator Script Generation

In [53]:
# Prompt template to generate teh narrator text
topic_template = """
You are to narrate a tutorial video about using an app. Be informative and do not make up things.
You are given a description about the app as well as the tutorial topic.
You will then be given the list of steps taken in the video alongside the timestamps where they start.
Your narration must abide by the timestamps.
Generate text for the narrator to read out to the viewer and nothing more.

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The steps(with timestamps) being performed in the video are:
```
{steps}
```
"""

In [54]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are a narrator with experience in making tech tutorial videos.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc", "tutorial_desc", "steps"]))

In [55]:
# Chain to write the script for the narrator
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
script_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='script')

In [56]:
# testing the scripts + prompt engineering

In [59]:
messages = [
    SystemMessage(
        content="You are a narrator with experience in making tech tutorial videos."
    ),
    HumanMessage(
        content="""
You are to narrate a tutorial video about using an app. Be informative and do not make up things.
You are given a description about the app as well as the tutorial topic.
You will then be given the list of steps taken in the video alongside the timestamps where they start.
Your narration must abide by the timestamps. If the time window is short, keep the text short aswell.
Generate text for the narrator to read out to the viewer and nothing more.

App details:
```
VITian is an application built for students of VIT to access their student data. 
We can use the app to find out examination allotments.
```

Tutorial Description:
```
This is a video tutorial on how to use the VITian app to check exam schedule and timings.
```

The steps(with timestamps) being performed in the video are:
```
[
    ("Open the VITN application", 4.8),
    ("Tap on the refresh button to get the latest data", 9.08),
    ("Open the sidebar by tapping on the sidebar icon", 15.2),
    ("Go into academics, then exam schedule", 21.04),
    ("Select your exam to view the exam venue, details, time and date", 23.4)
]
```
"""
    ),
]

In [60]:
op2 = chat(messages)

In [61]:
op2.content

'[0:00-4.7]\n"Hello everyone, welcome to our tutorial on how to use the VITian app to check your exam schedule and timings. This app is specifically designed for students of VIT to access their student data, including exam allotments. Let\'s get started."\n\n[4.8-9.07]\n"First, open the VITian application on your device. You\'ll see the home screen with various options."\n\n[9.08-15.19]\n"Next, tap on the refresh button located at the top right corner of the screen. This will ensure that you have the latest data and updates from the university."\n\n[15.2-21.03]\n"Now, open the sidebar by tapping on the sidebar icon, usually represented by three horizontal lines, located at the top left corner of the screen."\n\n[21.04-23.39]\n"Once the sidebar is open, navigate to the \'Academics\' section. Under this, you will find the \'Exam Schedule\' option. Tap on it to proceed."\n\n[23.4-End]\n"Here, you will see a list of your exams. Select the exam you want to view. This will display the exam v

# Generate the audio